# Testing

A notebook to test the implementation

In [1]:
import gzip
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
print(tf.config.list_physical_devices('GPU'))

/Users/customer/miniforge3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## 0 Preprocessing

For testing, we use the `MNIST` data. We first extract the data from `gzip` file.

In [2]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return (tf.cast(image, tf.float32)/255.0 , label)  

def preProcess(dset, batch_size):
  '''Pre-processes the dataset.'''
  dset = dset.map(normalize_img)
  dset = dset.shuffle(len(dset))
  dset = dset.batch(batch_size)
  return dset
  
# prepare data
def get_imgs(dir):
    with gzip.open(dir, 'r') as f:
        # first 4 bytes (some number)
        _ = int.from_bytes(f.read(4), 'big')
        # no. of images
        num_imgs = int.from_bytes(f.read(4), 'big')
        # row count
        row_cnt = int.from_bytes(f.read(4), 'big')
        # column count
        col_cnt = int.from_bytes(f.read(4), 'big')

        img_data = f.read()
        images = np.frombuffer(img_data, dtype=np.uint8).\
            reshape((num_imgs, row_cnt, col_cnt))
        return images

def get_labels(dir):
    with gzip.open(dir, 'r') as f:
        _ = int.from_bytes(f.read(4), 'big')
        label_cnt = int.from_bytes(f.read(4), 'big')
        print(label_cnt)
        label_data = f.read()
        labels = np.frombuffer(label_data, dtype=np.uint8)
        return labels

We now make `dataset`. Note that we used the `tf.data.Dataset.from_tensor_slices` to create dataset from a tuple of numpy arrays; if they were instead lists, then there will be an error

In [3]:
def getLocalDset(batch_size = 128):
    imgs     = get_imgs('data/train-images-idx3-ubyte.gz')
    labels   = get_labels('data/train-labels-idx1-ubyte.gz')
    split    = int(0.75*len(imgs))
    ds_train = tf.data.Dataset.from_tensor_slices((imgs[:split], labels[:split]))
    ds_test  = tf.data.Dataset.from_tensor_slices((imgs[split:], labels[split:]))
    print(len(ds_train))
    print(len(ds_test))
    return preProcess(ds_train, batch_size), preProcess(ds_test, batch_size)

We can also get the data by downloading

In [4]:
def fetch_img(ds, batch_size = 128):
  ds = ds.cache()
  # ds = ds.shuffle(ds_info.splits['train'].num_examples)
  ds = ds.batch(batch_size)
  # ds = ds.prefetch(tf.data.AUTOTUNE)
  return ds

def getRemoteDset(batch_size = 128, name = 'MNIST'):

  '''Prepares the MNIST dataset for training.'''  

  if name == 'MNIST':
    (ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    # split=['test', 'train[0%:10%]','train[10%:]'],
    split = ['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info = True
    )

  ds_train = ds_train.map(normalize_img, 
                          num_parallel_calls=tf.data.AUTOTUNE)
  ds_test = ds_test.map(normalize_img, 
                          num_parallel_calls=tf.data.AUTOTUNE)
  ds_train = fetch_img(ds_train, batch_size)
  ds_test = fetch_img(ds_test, batch_size)

  return (ds_train, ds_test), ds_info

(ds_train, ds_test), ds_info = getRemoteDset()

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-08-24 19:21:07.996315: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-24 19:21:07.996518: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## 1 Running experiment

First define some hyperparameters

In [5]:
layers = [300, 100]

activation = 'relu'

lr = 0.001

patience = 3

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

metrics = tf.keras.metrics.SparseCategoricalAccuracy()    

train_loss = tf.keras.metrics.Mean

train_acc = tf.keras.metrics.SparseCategoricalAccuracy  

epochs = 1 # use a much bigger int

num_pruning = 1 # use 5 or more  

step_perc = 0.5

In [6]:
from pruning import *
from models import *

model_params = {'input_shape': (28, 28, 1),
                'output_shape': 10,
                'layers': layers,
                'activation': 'relu',
                'BatchNorm':False,
                'Dropout': [0.5,0.5,0.5],
                'optimizer': tf.keras.optimizers.Adam(lr),
                'loss_fn': loss_fn,
                'metrics': metrics
                }

train_params = {'train_loss': train_loss,
                'train_acc': train_acc,
                'patience': 3
                }

prune_exp = pruning(ds_train, ds_test, model_params,
                    train_params,
                    epochs, num_pruning, step_perc)
                    
# prune_exp.test_training()
# prune_exp.test_model()

In [7]:
prune_exp.prune()


 
 Iterative pruning round: 0 
 


 
 After pruning, the total no. of nonzero weights is: 266200 
 


 Start original model training. 

Epoch 0 for original


2022-08-24 19:21:10.488167: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-24 19:21:10.664992: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-24 19:21:13.674668: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-24 19:21:13.798093: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


OG train acc 0.8447166681289673


2022-08-24 19:21:14.202212: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Original model accuracy 0.9488000273704529 


 
 After pruning, the total no. of nonzero weights is: 133100 
 


 Start Lottery ticket training 

Epoch 0 for lottery ticket


2022-08-24 19:21:14.428535: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-24 19:21:17.235318: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-24 19:21:17.344765: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Ticket train acc 0.4596500098705292 



2022-08-24 19:21:17.569792: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Lottery ticket accuracy 0.8523000478744507 


 Lottery ticket training finished. Highest Accuracy 0.8523000478744507 

INFO:tensorflow:Assets written to: saved_models/ticket_acc_0.8523000478744507 pruning round_0/assets


INFO:tensorflow:Assets written to: saved_models/ticket_acc_0.8523000478744507 pruning round_0/assets


In [8]:
prune_exp.removeLogs()

Removing 20220824-191036
Removing 20220824-191031
Removing 20220824-191630
Removing 20220824-191329
Removing 20220824-192110
Removing 20220824-190623
Removing 20220824-185646


In [9]:
prune_exp.removeModels()

Removing ticket_acc_0.8523000478744507 pruning round_0
